### 11. Spark ML

This notebook will introduce Spark ML and its API.

#### Correlation calculation


In [4]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder.appName("SparkML").getOrCreate()
import spark.implicits._

In [5]:
import org.apache.spark.ml.stat.Correlation
import org.apache.spark.ml.linalg.{Vectors, Matrix}
import org.apache.spark.sql.Row

val data = Seq(
  Vectors.sparse(4, Seq((0, 1.0), (3, -2.0))),  
  Vectors.dense(4.0, 5.0, 0.0, 3.0),
  Vectors.dense(6.0, 7.0, 0.0, 8.0),
  Vectors.sparse(4, Seq((0, 9.0), (3, 1.0)))
)

val df = data.map(Tuple1.apply).toDF("features")
val Row(coeff1: Matrix) = Correlation.corr(df, "features").head
println("Pearson correlation matrix:\n" + coeff1.toString)
val Row(coeff2: Matrix) = Correlation.corr(df, "features", "spearman").head
println("\nSpearman correlation matrix:\n" + coeff2.toString)

Pearson correlation matrix:
1.0                   0.055641488407465814  NaN  0.4004714203168137  
0.055641488407465814  1.0                   NaN  0.9135958615342522  
NaN                   NaN                   1.0  NaN                 
0.4004714203168137    0.9135958615342522    NaN  1.0                 

Spearman correlation matrix:
1.0                  0.10540925533894532  NaN  0.40000000000000174  
0.10540925533894532  1.0                  NaN  0.9486832980505141   
NaN                  NaN                  1.0  NaN                  
0.40000000000000174  0.9486832980505141   NaN  1.0                  




Let us understand the above piece of code step by step. There are two ways to create vectors, first one is dense and another is sparse. In a dense vector we specify all n elements and its values. The dense vector is simply created as ``Vectors.dense(4.0, 5.0, 0.0, 3.0)`` where as sparse is created as ``Vectors.sparse(4, Seq((0, 1.0), (3, -2.0)))`` wgere the first number if the number of elements/dimension in the vector, and the ``Seq`` given is a tuple of index, (value pair). Thus ``Vectors.sparse(4, Seq((0, 1.0), (3, -2.0)))`` is same as ``Vectors.dense(1.0, 0.0, 0.0, -2.0)`` as seen below

In [6]:
Vectors.sparse(4, Seq((0, 1.0), (3, -2.0))).toDense

[1.0,0.0,0.0,-2.0]


The correlation matrix is is always symmetric across diagonal with diagonal values being 1 as the corelation of a vector with itself is always 1. The matrix is symmetric across the diagonal that is element (0, 1) is same as (1, 0), (0, 2) same as (2, 0) and so on.

The formula for pearson correlation is 

$p\:=\:\frac{n\sum{xy} - (\sum{x})(\sum{y})}{\sqrt{[n\sum{x^2} - (\sum{x})^2][n\sum{y^2} - (\sum{y})^2]}}$

The correlation between two lists (1.0, 4.0, 6.0, 9.0) and (0.0, 5.0, 7.0, 0.0)
as per [this](http://calculator.vhex.net/calculator/statistics/pearson-correlation) URL is expected to be 0.055641.

The following code snippet calculates this pearson correlation between two list of doubles of equal length.

In [7]:
import scala.math.sqrt

def pearsonCorrelation(x:List[Double], y: List[Double]): Double = {
    val n = x.length
    val sumx = x.reduce(_ + _)
    val sumxsquare = x.map(e => e * e).reduce(_ + _)
    val sumy = y.reduce(_ + _)
    val sumysquare = y.map(e => e * e).reduce(_ + _)
    val sumxy = (x zip y).map{case (l, r) => l * r}.reduce(_ + _)
    val numerator = (n * sumxy) - (sumx * sumy)
    val denominator = (n * sumxsquare - sumx * sumx) * (n * sumysquare - sumy * sumy)
    numerator / math.sqrt(denominator)
}

val v1 = List(1.0, 4.0, 6.0, 9.0)
val v2 = List(0, 5.0, 7.0, 0)
println("Pearson coefficient between v1 and v2 is " + pearsonCorrelation(v1, v2))

Pearson coefficient between v1 and v2 is 0.055641488407465724


In [8]:
// Alternate implementation of above but by calculating the mean value first. 
// The above implementation doesn't need to calculate
val n = v1.length
val meanx = v1.reduce(_ + _) * 1.0 / n
val meany = v2.reduce(_ + _) * 1.0 / n

val numerator = (v1 zip v2).map{
 case (e1, e2) => (e1 - meanx) * (e2 - meany)
}.reduce(_ + _)

val sumxsquare = v1.map( e => (e - meanx) * (e - meanx)).reduce(_ + _)
val sumysquare= v2.map(e => (e - meany) * (e - meany)).reduce(_ + _)
val denominator = math.sqrt(sumxsquare) * math.sqrt(sumysquare)


println("Covariance between x and y is " + numerator / denominator)

Covariance between x and y is 0.055641488407465724



The vectors defined in ``data`` for which we computed the pearson coefficient are

```
[1.0,0.0,0.0,-2.0]
[4.0,5.0,0.0,3.0]
[6.0,7.0,0.0,8.0]
[9.0,0.0,0.0,1.0]
```

The line ``val df = data.map(Tuple1.apply).toDF("features")`` creates a ``DataFrame``. The ``toDF(columnName)`` is a a function that can be applied to ``Seq[Tuple{n}]``. This is an implicit function that lets us convert tuples to data frame which we get by the import ``scala.implicits._``. Each tuple in the sequence becomes a row and each element of the tuple becomes a column. To achieve this, we need to convert ``Seq[Vector]`` to ``Seq[Tuple1]`` using ``map`` so that we can convert the ``Seq[Tuple1]`` to a ``DataFrame`` with one column which we will call features.

The code ``Correlation.corr(df, "features")`` computes the correlation matrix. The return value of this call is a ``DataFrame`` with one row and one column. The name of the column is ``pearson(<nae of the original column in dataset>)`` and the type of the value is a ``Matrix``. The code ``val Row(coeff1: Matrix) = Correlation.corr(df, "features").head`` is a one liner to get the first and only row of this ``DataFrame`` and assign the value of the ``Matrix`` in it to the variable ``coeff1``.

The parameters of the ``corr`` function are the ``DataFrame`` instance, the name of the column in the ``DataFrame`` and an optional third parameter for the correlation method, which defaults to ``pearson``. Only valid value for now is ``spearman``.


---

#### ChiSquared Test

We will now look at Hypothesis testing which tests whether the result we get is stastically significant or not. We will see Pearsons Chi-Squared tests in this section. First a code sample and the result we get




In [23]:
import org.apache.spark.ml.stat.ChiSquareTest
import org.apache.spark.ml.linalg.Vector

val data = Seq(
  (0.0, Vectors.dense(0.5, 10.0)),
  (0.0, Vectors.dense(1.5, 20.0)),
  (1.0, Vectors.dense(1.5, 30.0)),
  (0.0, Vectors.dense(3.5, 30.0)),
  (0.0, Vectors.dense(3.5, 40.0)),
  (1.0, Vectors.dense(3.5, 40.0))
)

val df = data.toDF("label", "features")
val chiDF = ChiSquareTest.test(df, "features", "label")
val chi = chiDF.head
println("Printing the ChiSquaredTest DataFrame")
chiDF.show(truncate = false)
println("pValues = " + chi.getAs[Vector](0))
println("degreesOfFreedom = " + chi.getSeq[Int](1).mkString("[", ",", "]"))
println("statistics = " + chi.getAs[Vector](2))

Printing the ChiSquaredTest DataFrame
+---------------------------------------+----------------+----------+
|pValues                                |degreesOfFreedom|statistics|
+---------------------------------------+----------------+----------+
|[0.6872892787909721,0.6822703303362126]|[2, 3]          |[0.75,1.5]|
+---------------------------------------+----------------+----------+

pValues = [0.6872892787909721,0.6822703303362126]
degreesOfFreedom = [2,3]
statistics = [0.75,1.5]



We will now derive the above statistics using plain RDDs to see how the calculation is performed step by step. Note that this is not necessarily the most efficient way, but it demonstrates the steps nevertheless. 

We start by creating a contingency matrix for the each feature. The number of rows of this matrix are same as the number of unique values of that feature and the number of columns is same as number of unique labels. The values in the matrix are same as the number of occurances for that feature, label combination. In our case we will build two contingency matrix for each feature. For the first feature the unique values are (0.5, 1.5, 3.5) and the second feature has the unique values (10, 20, 30, 40). The number of columns are 2 in both cases for the labels (1, 0).

As an illustration, the matrix for first feature would be as follows. We have also added the row sum and the col sum for the following matrix

|               | **0**          | **1**  | **sum**|
|:-------------: |:-------------:|:-------------: |:-------------: |
| **0.5**      |  1| 0 |1|
| **1.5**      | 1      |   1 |2|
| **3.5**      | 2     |    1 |3|
| **sum**      | 4    |   2  ||


Let us give the input as a List of ``Tuples[(Double, Double)]``. This is for one dimension of the feature vector.

In [75]:
val inputList = List((0.5, 0), (1.5, 0), (1.5, 1), (3.5, 0), (3.5, 0), (3.5, 1))
val inputs = sc.parallelize(inputList)
val freq = inputs.map(i => (i, 1)).reduceByKey(_ + _).collectAsMap
val rowSum = inputs.map(i => (i._1, 1)).reduceByKey(_ + _).collectAsMap
val colSum = inputs.map(i => (i._2, 1)).reduceByKey(_ + _).collectAsMap
val inputSize = inputList.size
val uniqueLabels = inputList.map(_._2).toSet
val uniqueFeatures = inputList.map(_._1).toSet
println("(Label, Feature) pairs counts are " + 
                freq.mkString("[", ", ", "]") + 
                "\nRow Sum are " + rowSum.mkString("[", ", ", "]") + 
                "\nCol Sum are " + colSum.mkString("[", ", ", "]") )

(Label, Feature) pairs counts are [(1.5,0) -> 1, (3.5,0) -> 2, (3.5,1) -> 1, (0.5,0) -> 1, (1.5,1) -> 1]
Row Sum are [3.5 -> 3, 1.5 -> 2, 0.5 -> 1]
Col Sum are [1 -> 2, 0 -> 4]



The above Map, Row Sum and Col Sum values are same as the the Matrix we saw earlier. The next step is to compute the expected value. The way we compute the expected value is compute the probability of each of the possible labels and then multiply the total number of occurances of the feature with these probability.

For example, in the above case, from Col Sum we have the frequency of label 0 and 1 is 0.67 (4 / 6) and 0.33 (2 / 6) respectively. Expected value for feature value 3.5 would be 2( 3 $\times$ 0.67) and 1( 3 $\times$ 0.33) for label 0 and 1 respectively

Following code computes the expected value of each feature value, label value combination.

In [87]:
val expectedValues = (for(f <- uniqueFeatures ; l <-  uniqueLabels) yield(f, l)) map {
    case (f, l) => ((f, l), 1.0 * rowSum(f) * colSum(l) / inputSize)
}
println("Expected values for possible (Label, Feature) pair are " + expectedValues)

Expected values for possible (Label, Feature) pair are Set(((1.5,0),1.3333333333333333), ((3.5,0),2.0), ((0.5,1),0.3333333333333333), ((0.5,0),0.6666666666666666), ((1.5,1),0.6666666666666666), ((3.5,1),1.0))



For calculating $\chi^2$ stat. We will implement the following

$\chi^2\:=\:\sum_{i=1}^{N}\frac{(O_i - E_i)^2}{E_i}$

Where

- $O_i$ = the number of observations of type i.
- N = Total Number of observations
- $E_i$ = Expected theoritical probability of type i


Degrees of freedom is simply computed as $(n_f - 1) \times (n_l - 1)$

Where 

- $n_f$: Number of unique feature values
- $n_l$: Number of unique labels

In [94]:
val stat = expectedValues.foldLeft(0.0){
    case(acc, (k, e)) =>
        val f = freq.getOrElse(k, 0)
        val diff = f - e
        acc + diff * diff / e        
}
val df = (uniqueFeatures.size - 1) * (uniqueLabels.size - 1)
println("Stat is " + stat + ", degrees of freedom are " + df)

Stat is 0.7500000000000001, degrees of freedom are 2




With the Stat and degrees of freedom calculated, we will now calculate the pValue. The above degrees of freedom and stat are in sync with what we calculated using SparkML.